## Scraping Multiple Thread Tribunews

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [19]:
jumlah_index = 50
threads_link = []
links = []

In [20]:
def scrape_links(page_number):
    url = f"https://www.tribunnews.com/2023/09?page={page_number}"
    try:
        response = requests.get(url, headers={'User-Agent': 'Chrome/117.0.0.0'})
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.find_all('li', {"class": "ptb15"})
        page_links = []
        for article in articles:
            header = article.find('h3', {"class": "fbo f16"})
            link = header.find('a')['href']
            links.append(link)
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data from {url}: {e}")   

In [21]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 0 links from page 10
Scraped 0 links from page 35
Scraped 0 links from page 13
Scraped 0 links from page 30
Scraped 0 links from page 3
Scraped 0 links from page 20
Scraped 0 links from page 28
Scraped 0 links from page 15
Scraped 0 links from page 18
Scraped 0 links from page 41
Scraped 0 links from page 11
Scraped 0 links from page 22
Scraped 0 links from page 9
Scraped 0 links from page 27
Scraped 0 links from page 19
Scraped 0 links from page 25
Scraped 0 links from page 7
Scraped 0 links from page 24
Scraped 0 links from page 23
Scraped 0 links from page 31
Scraped 0 links from page 48
Scraped 0 links from page 8
Scraped 0 links from page 26
Scraped 0 links from page 1
Scraped 0 links from page 32
Scraped 0 links from page 6
Scraped 0 links from page 39
Scraped 0 links from page 12
Scraped 0 links from page 40
Scraped 0 links from page 34
Scraped 0 links from page 47
Scraped 0 links from page 16
Scraped 0 links from page 49
Scraped 0 links from page 46
Scraped 0 links from

In [22]:
results = []
def scrape_url(url):
    response = requests.get(url, headers={'User-Agent': 'Chrome/117.0.0.0'})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Judul Berita
        title_elem = soup.find('h1', {"id": "arttitle"})
        if title_elem:
            title_text = title_elem.text.strip()
        else:
            title_text = "Title not found"
        # Author berita
        author_elem = soup.find('div', {"id": "penulis"})
        if author_elem:
            penulis = author_elem.find('a')
            author_text =penulis.text.strip()
        else:
            author_text = "Author not found"
           # Editor Berita
        editor_elem = soup.find('div', {"id": "editor"})
        
        if editor_elem:
            editors = editor_elem.find('a')
            editor_text = editors.text.strip()
        else:
            editor_text = "editor not found" 
        
        # tanggal berita
        date_elem = soup.find('time', {"class": "grey"})
        if date_elem:
            date_text = date_elem.text.strip()
        else:
            date_text = "Date not found"
        #     # Category berita
        category_elements = soup.find_all('span',{"itemprop": "name"})
        if category_elements:
            category_text= category_elements[2].get_text()
        else:
            category_text = "Category not found"
        ## Content Berita
        body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize"})
        
        if body_elem:
            content_elem = body_elem.find_all('p')
            content_text = ""
            for p in content_elem:
                content_text += p.text.strip() + "\n"
            
            if content_text.strip():
                content_text=content_text
            else:
                content_text ="Content not found"
        else:
          content_text ="Content not found"

        results.append({'title': title_text,
                        'author' : author_text,
                        'editor' : editor_text,
                        'category':category_text,
                        'date': date_text,
                        'content' : content_text,
                        'link' : url})
    else:
        print(f"Failed to retrieve data from {url}")

In [23]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Exception in thread Thread-171 (scrape_url):
Traceback (most recent call last):
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 975, in run
Exception in thread Thread-176 (scrape_url):
Traceback (most recent call last):
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 975, in run
Exception in thread Thread-186 (scrape_url):
Traceback (most recent call last):
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/krisna/anaconda3/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_31296/218346612.py", line 38, in scrape_url
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_31296/218346612.py", li

In [24]:
df = pd.DataFrame(results)

df.head(10)

,title,author,category,date,content,link
0,Tampilan iPhone 15 Pro Max Dikeluhkan Cacat De...,Namira Yunia Lestanti,Gadget,"Senin, 25 September 2023 17:18 WIB",Laporan Wartawan Tribunnews.com Namira Yunia L...,https://www.tribunnews.com/techno/2023/09/25/t...
1,"Patuhi Perintah Ketum PSSI, Persis Lepas Sanan...",Author not found,Lainnya,"Senin, 25 September 2023 17:26 WIB","TRIBUNNEWS.COM, SOLO- Manajemen Persis Solo me...",https://www.tribunnews.com/superskor/2023/09/2...
2,Kaderisasi PSI Dipertanyakan Imbas Usulan Kaes...,Faryyanida Putwiliani,Politik,"Senin, 25 September 2023 17:20 WIB",TRIBUNNEWS.COM - Usulan putra bungsu Presiden ...,https://www.tribunnews.com/nasional/2023/09/25...
3,"Bisnis Keripik Pisang Menggiurkan, Wong Kito G...",Reza Deni,Sumatera,"Senin, 25 September 2023 17:25 WIB","Laporan Reporter Tribunnews.com, Reza Deni\nTR...",https://www.tribunnews.com/regional/2023/09/25...
4,Atlet Menembak Muhammad Sejahtera Raih Emas Pe...,Abdul Majid,Olahraga Lainnya,"Senin, 25 September 2023 17:21 WIB","TRIBUNNEWS.COM, JAKARTA - Atlet menembak Indon...",https://www.tribunnews.com/sport/2023/09/25/at...
5,"Fakta Kaesang Gabung PSI: Akui Punya Kesamaan,...",Faryyanida Putwiliani,Politik,"Minggu, 24 September 2023 20:24 WIB",TRIBUNNEWS.COM - Putra Presiden Joko Widodo (J...,https://www.tribunnews.com/nasional/2023/09/24...
6,Ada Kabar Baik Antara M.A.C dengan Pihak Pesta...,Fauzi Nur Alamsyah,Musik,"Minggu, 24 September 2023 20:41 WIB","Laporan Wartawan Tribunnews.com, Fauzi Alamsya...",https://www.tribunnews.com/seleb/2023/09/24/ad...
7,"Format Surat Pernyataan 5 Poin CPNS BIN 2023, ...",Muhammad Alvian Fakka,Umum,"Senin, 25 September 2023 17:11 WIB",TRIBUNNEWS.COM - Berikut format surat pernyata...,https://www.tribunnews.com/nasional/2023/09/25...
8,Jadwal Piala Carabao 2023: Chelsea Rawan Tersi...,Dwi Setiawan,Inggris,"Senin, 25 September 2023 17:13 WIB",TRIBUNNEWS.COM - Simak jadwal Piala Carabao 20...,https://www.tribunnews.com/superskor/2023/09/2...
9,Persis Solo Tegaskan Lepas Sananta ke Timnas I...,Alfarizy Ajie Fadhillah,Lainnya,"Senin, 25 September 2023 17:16 WIB","Laporan Wartawan Tribunnews.com, Alfarizy AF\n...",https://www.tribunnews.com/superskor/2023/09/2...


save_date

In [25]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/tribunnews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/tribunnews_2023-09-25_21-27-51.xlsx
